In [103]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import random

In [104]:
import warnings
warnings.filterwarnings("ignore")

In [105]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [106]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [107]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [108]:
answers = {}

In [109]:
# Some data structures that will be useful

In [110]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [111]:
len(allRatings)

200000

In [112]:
allRatings[0]

('u67805239', 'b61372131', 4)

In [113]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [114]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
predictions.close()

In [115]:
##################################################
# Read prediction                                #
##################################################

In [116]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break



In [117]:
### Question 1

In [118]:
booksPerUserValid = defaultdict(set)
usersPerBookAllRatings = defaultdict(set)

for u, b, r in allRatings:
    # get sets of users that have read book
    usersPerBookAllRatings[b].add(u)

# get set of books from ALL RATINGS
set_of_books = set([b for b in usersPerBookAllRatings])


for d in ratingsValid:
    # get sets of books for each user that they've read
    booksPerUserValid[d[0]].add(d[1])

print(f"books per user valid set: {[(u, books) for ind, (u, books) in enumerate(booksPerUserValid.items()) if ind < 2]}")
print(f"Users per book in all rating: {[(b, users) for ind, (b, users) in enumerate(usersPerBookAllRatings.items()) if ind < 2]}")
print(f"length of ratingValid: {len(ratingsValid)}")
print(f"booksPerUserValid length: {len(booksPerUserValid)}")

books per user valid set: [('u59070515', {'b55084829'}), ('u05014036', {'b49553867'})]
Users per book in all rating: [('b61372131', {'u01824015', 'u81545593', 'u10749976', 'u64965360', 'u69544334', 'u73816851', 'u58944461', 'u42850585', 'u39594275', 'u94503926', 'u30118092', 'u87998402', 'u63351603', 'u30806009', 'u95654536', 'u99062581', 'u82434460', 'u91947449', 'u38854826', 'u20491231', 'u51394014', 'u22497171', 'u89732246', 'u32610240', 'u46260999', 'u21325396', 'u03384038', 'u87366750', 'u26839529', 'u16132576', 'u79151084', 'u99811205', 'u36256752', 'u93815296', 'u48391630', 'u92278115', 'u98980325', 'u94309193', 'u18822429', 'u12983626', 'u62772540', 'u24156255', 'u80458214', 'u59157206', 'u71834505', 'u07240369', 'u47580089', 'u09441867', 'u20409355', 'u11499326', 'u26716074', 'u20496815', 'u62421718', 'u49555043', 'u43490914', 'u80993836', 'u53489985', 'u55760432', 'u34251370', 'u40248621', 'u84127839', 'u00111518', 'u11287990', 'u80414310', 'u54632797', 'u84677719', 'u3523806

In [119]:
notRead_valid_set = []

for d in ratingsValid:
    # get the books that user have not read
    diff = set_of_books.difference(booksPerUserValid[d[0]])

    # add not_read books to existing validation set 
    notRead_valid_set.append((d[0], list(diff)[random.randint(0, len(diff)-1)]))
    # notRead_valid_set[d[0]].append(list(diff)[random.randint(0, len(diff)-1)]) # get random book for user


print(f"List (user, book) entries of unread books by user: {[(u, book) for ind, (u, book) in enumerate(notRead_valid_set) if ind < 2]}")
print(len(notRead_valid_set))

List (user, book) entries of unread books by user: [('u59070515', 'b08643931'), ('u05014036', 'b78420438')]
10000


In [120]:
# adding to current validation set
print(f"before length: {len(ratingsValid)}")
ratingsValid_q1 = ratingsValid
for u, b in notRead_valid_set:
    ratingsValid_q1.append((u,b))

print(f"after length: {len(ratingsValid)}")

before length: 10000
after length: 20000


In [121]:
def accuracy(pred, y):
    TP_ = numpy.logical_and(pred, y)
    FP_ = numpy.logical_and(pred, numpy.logical_not(y))
    TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(y))
    FN_ = numpy.logical_and(numpy.logical_not(pred), y)

    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)

    acc = (TP + TN)/len(pred)
    return acc

# Evaluate performance of baseline model on validation set
y_q1 = [1]*10000 + [0]*10000
ypred_q1 = []
count_exist = 0
for d in ratingsValid_q1:
    if d[1] in return1:
        ypred_q1.append(1)
    else:
        ypred_q1.append(0)

acc1 = accuracy(ypred_q1, y_q1)
print(acc1)

0.71105


In [122]:
answers['Q1'] = acc1

In [123]:
assertFloat(answers['Q1'])

In [124]:
### Question 2

In [125]:
# reset rating valid and other data sets
# ratingsTrain = allRatings[:190000]
# ratingsValid = allRatings[190000:]
# ratingsPerUser = defaultdict(list)
# ratingsPerItem = defaultdict(list)
# for u, b, r in ratingsTrain:
#     ratingsPerUser[u].append((b, r))
#     ratingsPerItem[b].append((u, r))

# add 

acc2 = 0
threshold = 2
for thres in [0.05*r for r in range(1, 20)]:
    return2 = set()
    count_q2 = 0
    ypred_q2 = []
    for ic, i in mostPopular:
        count_q2 += ic
        return2.add(i)
        if count_q2 > totalRead*thres: break
    for d in ratingsValid:
        if d[1] in return2:
            ypred_q2.append(1)
        else:
            ypred_q2.append(0)
    curr_acc = accuracy(ypred_q2, y_q1)
    if curr_acc > acc2:
        acc2 = curr_acc
        threshold = thres

In [126]:
answers['Q2'] = [threshold, acc2]
print(threshold)
print(acc2)

0.75
0.75375


In [127]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

## Question 3

In [128]:
### Question 3/4

In [129]:
def Jaccard(s1, s2):
    numerator = len(s1.intersection(s2))
    denominator = len(s1.union(s2))
    if denominator == 0:
        return 0
    return numerator/denominator


In [130]:
# setting up data structures from training and validation data

booksPerUser_train = defaultdict(set)
booksPerUser_valid = defaultdict(set)
usersPerBook_train = defaultdict(set)
usersPerBook_valid = defaultdict(set)

for u, b, r in ratingsTrain:
    booksPerUser_train[u].add(b)
    usersPerBook_train[b].add(u)

for d in ratingsValid:
    booksPerUser_valid[d[0]].add(d[1])
    usersPerBook_valid[d[1]].add(d[0])


In [131]:
# for b in usersPerBook_valid:
#     sim = Jaccard(usersPerBook_train[u], usersPerBook_valid[u])
#     if sim > 0.5:
#         pass
acc3 = 0 
ypred_q3 = []
for d in ratingsValid:
    # get similaritiy for all pairs in validation set
    max_sim = 0                                                  # d[1] = b' = books from training set
    for b in booksPerUser_train[d[0]]:
        if b == d[1]: continue                                  # b = books from validation set, using the user in validation set
        sim = Jaccard(usersPerBook_train[b], usersPerBook_train[d[1]])  # compute jaccard similarity between users in train for books in validation set and training set
        if max_sim < sim:
            max_sim = sim
    if max_sim > 0:
        ypred_q3.append(1)
    else:
        ypred_q3.append(0)


In [132]:
acc3 = accuracy(ypred_q3, y_q1)
print(acc3)

0.6823


## Question 4

In [133]:
count_q4 = 0
return_q4 = set()
for ic, i in mostPopular:
    count_q4 += ic
    return_q4.add(i)
    if count_q4 > totalRead*0.75:           # threshold from Q2
        break

ypred_q4 = []
for d in ratingsValid:
    # get similaritiy for all pairs in validation set
    # d[1] = b' = books from training set
    max_sim = 0
    for b in booksPerUser_train[d[0]]:
        if b == d[1]:
            # b = books from validation set, using the user in validation set
            continue
        # compute jaccard similarity between users in train for books in validation set and training set
        sim = Jaccard(usersPerBook_train[b], usersPerBook_train[d[1]])
        if max_sim < sim:
            max_sim = sim
    if max_sim > 0 and d[1] in return_q4:
        ypred_q4.append(1)
    else:
        ypred_q4.append(0)


In [134]:
acc4 = accuracy(ypred_q4, y_q1)

In [135]:
answers['Q3'] = acc3
answers['Q4'] = acc4
print(f"acc3: {acc3}")
print(f"acc4: {acc4}")

acc3: 0.6823
acc4: 0.74295


In [136]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [173]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    max_sim = 0
    for b2 in booksPerUser_train[u]:
        if b2 == b:
            # b = books from validation set, using the user in validation set
            continue
        # compute jaccard similarity between users in train for books in validation set and training set
        sim = Jaccard(usersPerBook_train[b2], usersPerBook_train[b])
        if max_sim < sim:
            max_sim = sim
    if max_sim > 0 and b in return_q4:
       predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")

predictions.close()

In [138]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [139]:
assert type(answers['Q5']) == str

In [140]:
##################################################
# Category prediction (CSE158 only)              #
##################################################

In [141]:
### Question 6

In [142]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

In [143]:
data[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [144]:
category_all_data = [d for d in data]
category_train_data = category_all_data[:90000]
category_valid_data = category_all_data[90000:]

wordCount_train_q6 = defaultdict(int)
totalWords = 0

In [145]:
punct  = string.punctuation
# stemmer = PorterStemmer()

In [146]:
# get unique words and its counts
for d in category_train_data:
    rev:string = d['review_text']
    rev = rev.lower()
    rev = [c for c in rev if not (c in punct)]
    rev = ''.join(rev)
    words = rev.strip().split()
    for w in words:
        wordCount_train_q6[w] += 1
        totalWords += 1

# get the top 1000 most common words
counts_q6 = [(wordCount_train_q6[w], w) for w in wordCount_train_q6]
counts_q6.sort()
counts_q6.reverse()

In [147]:
answers['Q6'] = counts_q6[:10]

In [148]:
assert [type(x[0]) for x in answers['Q6']] == [int]*10
assert [type(x[1]) for x in answers['Q6']] == [str]*10

In [149]:
### Question 7

In [150]:
# get top 1000 unique words 
common1000 = [w for count, w in counts_q6[:1000]]
wordId_q7 = dict(zip(common1000, range(len(common1000))))
word_set_q7 = set(common1000) # build dictionary
len(word_set_q7)


1000

In [151]:
# iterate through entire dataset and count number of top 1000 unique words per review
def feature(datum):
    # Iterate through a review at a time
    feat = [0]*len(word_set_q7)
    rev = datum['review_text']
    rev = [c for c in rev if not (c in punct)]
    rev = ''.join(rev)
    words = rev.strip().split()
    for w in words:
        # check each word in review
        if not (w in word_set_q7): continue
        feat[wordId_q7[w]] += 1
    feat.append(1)                          # constant 
    return feat

X_q7 = [feature(d) for d in data] # a list_a of list_b. len(X) = len(y)
y = [d['genreID'] for d in data]

print(X_q7[0][:10])
print(y[:10])


[0, 2, 2, 0, 0, 0, 1, 0, 1, 0]
[3, 1, 4, 2, 2, 4, 2, 0, 4, 2]


In [152]:
X_q7train = X_q7[:9*len(X_q7)//10]
ytrain = y[:9*len(y)//10]
X_q7valid = X_q7[9*len(X_q7)//10:]
yvalid = y[9*len(y)//10:]

In [153]:
model = linear_model.LogisticRegression(C=1.0)
model.fit(X_q7train, ytrain)
ypred_valid_q7 = model.predict(X_q7valid)

acc7 = accuracy(yvalid, ypred_valid_q7)

In [154]:
answers['Q7'] = acc7
print(acc7)

0.9344


In [155]:
assertFloat(answers['Q7'])

In [156]:
### Question 8

In [157]:
acc8 = acc7
for c in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
    model_q8 = linear_model.LogisticRegression(C=c)
    model_q8.fit(X_q7train, ytrain)
    ypred_valid_q8 = model_q8.predict(X_q7valid)
    curr_acc = accuracy(yvalid, ypred_valid_q8)
    if curr_acc > acc8:
        acc8 = curr_acc
        print(f"acc8: {acc8}")
        print(f"c: {c}")

acc8: 0.9361
c: 0.01


In [158]:
# def feature(datum, common_words, word_set):
#     # Iterate through a review at a time
#     feat = [0]*len(word_set_q7)
#     rev = datum['review_text']
#     rev = [c for c in rev if not (c in punct)]
#     rev = ''.join(rev)
#     words = rev.strip().split()
#     for w in common1000:
#         # check each word in review
#         if not (w in word_set_q7):
#             continue
#         feat[wordId_q7[w]] += 1
#     feat.append(1)                          # constant
#     return feat


In [159]:
# for size in [2000, 3000, 4000, 5000, 6000]:
#     # get top {size} unique words
#     common_words = [w for count, w in counts_q6[:size]]
#     wordId_q8 = dict(zip(common_words, range(len(common_words))))
#     word_set_q8 = set(common_words)  # build dictionary
#     len(word_set_q7)
#     for 


In [160]:
answers['Q8'] = acc8
print(acc8)

0.9361


In [161]:
assertFloat(answers['Q8'])

In [162]:
# Run on test set

In [165]:
# Build model
model_submission = linear_model.LogisticRegression(C=0.01)
model_submission.fit(X_q7train, ytrain)

LogisticRegression(C=0.01)

In [181]:
data_cat = []

for d in readGz("test_Category.json.gz"):
    data_cat.append(d)

print(data_cat[0])

{'user_id': 'u77355739', 'review_id': 'r70788666', 'rating': 4, 'review_text': "love it \n it's that kind of crime-mystery novel that anyone can : read , love and understand . \n my favorite part is in the first chapter when Mr. quin help the old man( i forget his name) to solve a long time closed mystery , and that is only the beginning", 'n_votes': 0}


In [182]:
reviewID= defaultdict(set)
genreCount = defaultdict(int)
genreIDtoGenre = defaultdict(int)
for d in data_cat:
    reviewID[d['review_id']] = d

for d in data:
    genreCount[d['genre']] += 1
    genreIDtoGenre[d['genreID']] = d['genre']

reviewId_list = [r_id for r_id in reviewID]
genre_list = [(count, genre) for genre, count in genreCount.items()]
genre_list.sort()
genre_list.reverse()

In [183]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if b in reviewId_list:
        X_submission = [feature(reviewID[b])]
        ypred_submit = model_submission.predict(X_submission)
        predictions.write(f"{u},{b},{genreIDtoGenre[ypred_submit[0]]}\n")
    else:
        predictions.write(f"{u},{b},{genre_list[0][1]}\n")
        print("not a known review")
    

predictions.close()

In [164]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()